In [1]:
# run flame speeds to see which species (and reactions) exist in highest concentration: focus on those

In [2]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
import soln2cti

import numpy as np

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import csv
import sys
import copy

In [ ]:
# load David's 2-BTP model
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG_no_halogens/no_halogens.inp'
RMG_dictionary_path = 'models/RMG_no_halogens/no_halogens_dict.txt'
RMG_transport_path = 'models/RMG_no_halogens/no_halogens_tran.dat'
RMG_cti_path = 'models/RMG_no_halogens/no_halogens.cti'

# RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)

In [29]:
# Load NIST model without halogens
NIST_truncated_cti_path = 'models/NIST_no_halogens/no_halogens.cti'
NIST_truncated_gas = ct.Solution(NIST_truncated_cti_path)

For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [30]:
# Load NIST 2-BTP model
# /scratch/westgroup/David/halogen_models/models/2-BTP/NIST/2-BTP_kinetics.cti
NIST_cti_path = 'models/NIST/2-BTP_kinetics.cti'
NIST_dictionary_path = 'models/NIST/species_dictionary.txt'
NIST_chemkin_path = 'models/NIST/pym_gas_copy.inp'

# NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path)
NIST_gas = ct.Solution(NIST_cti_path)

For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [ ]:
# RMG flame speed
print("Running Cantera Version: " + str(ct.__version__))


T0 = 298
P0 = ct.one_atm
gas = RMG_gas

equiv_ratios = [1.0]

# can use below to only test one vol_frac
vol_frac_list = [0.0]

# Define stoichiometric coefficients for methane combustion reaction
v_fuel = 1.0
v_oxidizer = 2.0
results = {}

for i, phi in enumerate (equiv_ratios):
    try:

        tol_ss = [1.0e-13, 1.0e-9]  # abs and rel tolerances for steady state problem
        tol_ts = [1.0e-13, 1.0e-9]  # abs and rel tie tolerances for time step function

        # calculate composition from equivalence ratio
        actual_ratio = phi * (v_fuel / v_oxidizer)
        x_O2 = 1.0
        x_CH4 = actual_ratio * x_O2
        x_N2 = 0.79 * (x_O2 / .21)
        total = x_O2 + x_CH4 + x_N2
        x_O2 = x_O2 / total
        x_CH4 = x_CH4 / total
        x_N2 = x_N2 / total
        print(f'Starting compositions: x_O2={x_O2}, x_CH4={x_CH4}, x_N2={x_N2}')
        mol_fractions = {'CH4(3)': x_CH4, 'O2(4)': x_O2, 'N2': x_N2}

        # Define starting concentrations in molar ratios
        gas.TPX = T0, P0, mol_fractions
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.flame.set_steady_tolerances(default=tol_ss)   # set tolerances
        flame.flame.set_transient_tolerances(default=tol_ts)
        # flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)
        flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
        flame.max_time_step_count = 900
        loglevel = 1
        if i != 0:
            d = f'./data/David_test_{equiv_ratios[i-1]}.csv'
            if os.path.exists(d):
                arr2 = ct.SolutionArray(gas)
                arr2.read_csv(d)
                flame.set_initial_guess(data=arr2)
                print(' initial guess has been set')
        # "False" stops the calculation from retrying over and over, thanks Chao 
        # flame.solve(loglevel=loglevel, auto=False)
        print("about to solve")
        flame.solve(loglevel=loglevel, auto=True)
        Su = flame.velocity[0]
        results[phi] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        # edited this here!! index=False
        df1.to_csv(f'./data/David_test_{phi}.csv', index=False)
    except Exception as e:
        print(f'********************passed volume fraction:{equiv_ratios[i]}, error: {e}*************************************')
        pass


equiv_ratios = list(results.keys())
flame_speeds = list(results.values())

print("equivalence ratios are:")
print(equiv_ratios)

print("flame speeds are:")
print(flame_speeds)


flame_speed_file = os.path.join('flame_speeds', f'flame_speeds_RMG.csv')
with open(flame_speed_file, 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(equiv_ratios)
    writers.writerow(flame_speeds)


In [14]:
# NIST no halogens flame speed
print("Running Cantera Version: " + str(ct.__version__))


T0 = 298
P0 = ct.one_atm


equiv_ratios = [1.0]

# can use below to only test one vol_frac
vol_frac_list = [0.0]

# Define stoichiometric coefficients for methane combustion reaction
v_fuel = 1.0
v_oxidizer = 2.0
results3 = {}

for i, phi in enumerate (equiv_ratios):
    try:

        tol_ss = [1.0e-13, 1.0e-9]  # abs and rel tolerances for steady state problem
        tol_ts = [1.0e-13, 1.0e-9]  # abs and rel tie tolerances for time step function

        # calculate composition from equivalence ratio
        actual_ratio = phi * (v_fuel / v_oxidizer)
        x_O2 = 1.0
        x_CH4 = actual_ratio * x_O2
        x_N2 = 0.79 * (x_O2 / .21)
        total = x_O2 + x_CH4 + x_N2
        x_O2 = x_O2 / total
        x_CH4 = x_CH4 / total
        x_N2 = x_N2 / total
        print(f'Starting compositions: x_O2={x_O2}, x_CH4={x_CH4}, x_N2={x_N2}')
        mol_fractions = {'CH4': x_CH4, 'O2': x_O2, 'N2': x_N2}

        # Define starting concentrations in molar ratios
        NIST_truncated_gas.TPX = T0, P0, mol_fractions
        width = 0.08
        NIST_truncated_flame = ct.FreeFlame(NIST_truncated_gas, width=width)
        NIST_truncated_flame.flame.set_steady_tolerances(default=tol_ss)   # set tolerances
        NIST_truncated_flame.flame.set_transient_tolerances(default=tol_ts)
        NIST_truncated_flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
        NIST_truncated_flame.max_time_step_count = 900
        
        # "False" stops the calculation from retrying over and over, thanks Chao 
        # flame.solve(loglevel=loglevel, auto=False)
        print("about to solve")
        NIST_truncated_flame.solve(loglevel=loglevel, auto=True)
        Su = NIST_truncated_flame.velocity[0]
        results3[phi] = Su
        sltn = NIST_truncated_flame.to_solution_array()
        df1 = sltn.to_pandas()

    except Exception as e:
        print(f'********************passed volume fraction:{equiv_ratios[i]}, error: {e}*************************************')
        pass


equiv_ratios = list(results3.keys())
flame_speeds = list(results3.values())

print("equivalence ratios are:")
print(equiv_ratios)

print("flame speeds are:")
print(flame_speeds)


flame_speed_file = os.path.join('flame_speeds', f'flame_speeds_RMG.csv')
with open(flame_speed_file, 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(equiv_ratios)
    writers.writerow(flame_speeds)


Running Cantera Version: 2.5.1
Starting compositions: x_O2=0.19004524886877827, x_CH4=0.09502262443438914, x_N2=0.7149321266968326
about to solve

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.424e-05      4.316
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003649      3.114
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.001039       4.43
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.156e-05      6.569
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     4.389e-05      5.861
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.001125      3.821
Attempt Newton solution of steady-state problem...  

Attempt Newton solution of steady-state problem...    success.

Problem solved on [74] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 27 28 29 38 39 
    to resolve HOC2H4O2 aC3H5 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [79] point grid(s).

..............................................................................
no new points needed in flame

******************* Solving with user-specified tolerances *******************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [79] point gr

In [31]:
# NIST Flame speed
print("Running Cantera Version: " + str(ct.__version__))


T0 = 298
P0 = ct.one_atm
gas2 = NIST_gas

equiv_ratios = [1.0]

# can use below to only test one vol_frac
vol_frac_list = [0.0]

# Define stoichiometric coefficients for methane combustion reaction
v_fuel = 1.0
v_oxidizer = 2.0
results2 = {}

for i, phi in enumerate (equiv_ratios):
    try:

        tol_ss = [1.0e-15, 1.0e-11]  # abs and rel tolerances for steady state problem
        tol_ts = [1.0e-15, 1.0e-11]  # abs and rel tie tolerances for time step function
        
#         tol_ss = [1.0e-13, 1.0e-9]  # abs and rel tolerances for steady state problem
#         tol_ts = [1.0e-13, 1.0e-9]  # abs and rel tie tolerances for time step function

        # calculate composition from equivalence ratio
        actual_ratio = phi * (v_fuel / v_oxidizer)
        x_O2 = 1.0
        x_CH4 = actual_ratio * x_O2
        x_N2 = 0.79 * (x_O2 / .21)
        total = x_O2 + x_CH4 + x_N2
        x_O2 = x_O2 / total
        x_CH4 = x_CH4 / total
        x_N2 = x_N2 / total
        print(f'Starting compositions: x_O2={x_O2}, x_CH4={x_CH4}, x_N2={x_N2}')
        mol_fractions2 = {'CH4': x_CH4, 'O2': x_O2, 'N2': x_N2}

        # Define starting concentrations in molar ratios
        gas2.TPX = T0, P0, mol_fractions2
        width = 0.08
        flame2 = ct.FreeFlame(gas2, width=width)
        flame2.flame.set_steady_tolerances(default=tol_ss)   # set tolerances
        flame2.flame.set_transient_tolerances(default=tol_ts)
        # flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)
        flame2.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
        flame2.max_time_step_count = 900
        loglevel = 1
        
        print("about to solve")
        flame2.solve(loglevel=loglevel, auto=True)
        Su = flame2.velocity[0]
        results2[phi] = Su
        sltn = flame2.to_solution_array()
        df1 = sltn.to_pandas()
        # edited this here!! index=False

    except Exception as e:
        print(f'********************passed volume fraction:{equiv_ratios[i]}, error: {e}*************************************')
        pass


equiv_ratios = list(results2.keys())
flame_speeds = list(results2.values())

print("equivalence ratios are:")
print(equiv_ratios)

print("flame speeds are:")
print(flame_speeds)


flame_speed_file = os.path.join('flame_speeds', f'flame_speeds_NIST.csv')
with open(flame_speed_file, 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(equiv_ratios)
    writers.writerow(flame_speeds)


Running Cantera Version: 2.5.1
Starting compositions: x_O2=0.19004524886877827, x_CH4=0.09502262443438914, x_N2=0.7149321266968326
about to solve

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.136e-05      4.316
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003649      3.123
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0006928      4.367
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.541e-05      6.405
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.194e-05       6.16
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003749      4.598
Attempt Newton solution of steady-state problem...  

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      4.333
Attempt Newton solution of steady-state problem...    success.

Problem solved on [58] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 
    to resolve C C2H C2H2 C2H3 C2H3CHO C2H4 C2H5 C2H6 C2O C3H3 C3H6 C3H8 C4H10 C4H2 C4H4 C4H6 C4H81 CH CH2 CH2* CH2CHO CH2CO CH2OCH2 CH2OH CH3 CH3CCH2 CH3CH2O CH3CHCH CH3CHO CH3O H2C4O H2CC HCCO HCCOH HCO HO2 HOC2H4O2 aC3H4 aC3H5 iC3H7 iC4H10 iC4H8 nC3H7 pC3H4 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [74] point gri

# Run Flame Speed Function

In [63]:
def run_flame_speed(gas):
    T0 = 298
    P0 = ct.one_atm
    equiv_ratios = [1.0]

    # can use below to only test one vol_frac
    vol_frac_list = [0.0]

    # Define stoichiometric coefficients for methane combustion reaction
    v_fuel = 1.0
    v_oxidizer = 2.0
    results = {}

    for i, phi in enumerate (equiv_ratios):
        try:

            tol_ss = [1.0e-13, 1.0e-9]  # abs and rel tolerances for steady state problem
            tol_ts = [1.0e-13, 1.0e-9]  # abs and rel tie tolerances for time step function

            # calculate composition from equivalence ratio
            actual_ratio = phi * (v_fuel / v_oxidizer)
            x_O2 = 1.0
            x_CH4 = actual_ratio * x_O2
            x_N2 = 0.79 * (x_O2 / .21)
            total = x_O2 + x_CH4 + x_N2
            x_O2 = x_O2 / total
            x_CH4 = x_CH4 / total
            x_N2 = x_N2 / total
            print(f'Starting compositions: x_O2={x_O2}, x_CH4={x_CH4}, x_N2={x_N2}')
            mol_fractions = {'CH4(3)': x_CH4, 'O2(4)': x_O2, 'N2': x_N2}

            # Define starting concentrations in molar ratios
            gas.TPX = T0, P0, mol_fractions
            width = 0.08
            flame = ct.FreeFlame(gas, width=width)
            flame.flame.set_steady_tolerances(default=tol_ss)   # set tolerances
            flame.flame.set_transient_tolerances(default=tol_ts)
            flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
            flame.max_time_step_count = 900
            loglevel = 1

            print("about to solve")
            flame.solve(loglevel=loglevel, auto=True)
            Su = flame.velocity[0]
            results[phi] = Su
            sltn = flame.to_solution_array()
            df1 = sltn.to_pandas()
            # edited this here!! index=False

        except Exception as e:
            print(f'********************passed volume fraction:{equiv_ratios[i]}, error: {e}*************************************')
            pass


    equiv_ratios = list(results.keys())
    flame_speeds = list(results.values())

    print("equivalence ratios are:")
    print(equiv_ratios)

    print("flame speeds are:")
    print(flame_speeds)
    
    return flame


In [64]:
# Create RMG model using their transport

# load David's 2-BTP model
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG_no_halogens/no_halogens.inp'
RMG_dictionary_path = 'models/RMG_no_halogens/no_halogens_dict.txt'
RMG_transport_path = 'models/RMG_no_halogens/no_halogens_tran.dat'
RMG_cti_path = 'models/RMG_no_halogens/no_halogens.cti'

# RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)
RMG_dict = rmgpy.chemkin.load_species_dictionary(RMG_dictionary_path)

# Load NIST model transport
NIST_truncated_cti_path = 'models/NIST_no_halogens/no_halogens.cti'
NIST_truncated_gas = ct.Solution(NIST_truncated_cti_path)
NIST_dict_path = 'models/NIST_no_halogens/no_halogens_dict.txt'
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dict_path)


RMG_species = RMG_gas.species()
NIST_species = NIST_truncated_gas.species()
for rmg_sp in RMG_species:
    for nist_sp in NIST_species:
        if RMG_dict[rmg_sp.name].is_isomorphic(NIST_dict[nist_sp.name]):
            rmg_sp.transport = nist_sp.transport


For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [65]:
transport_flame = run_flame_speed(RMG_gas)

Starting compositions: x_O2=0.19004524886877827, x_CH4=0.09502262443438914, x_N2=0.7149321266968326
about to solve

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      3.452
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001825      5.423
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     3.247e-05      6.258
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0005548      3.956
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01422      3.102
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
grid refi

Attempt Newton solution of steady-state problem...    success.

Problem solved on [77] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 23 24 25 26 
    to resolve CH3O(28) CH3OO(201) 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [81] point grid(s).

..............................................................................
no new points needed in flame

******************* Solving with user-specified tolerances *******************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [81] point 

In [62]:
transport_flame.velocity[0]

0.2488300066590451

In [45]:
RMG_species[0].transport = RMG_species[1].transport

In [47]:
 RMG_gas.species()[0].transport.geometry

'atom'

In [10]:
flame2.gas.TPX = 298.0, ct.one_atm, mol_fractions2

In [26]:
flame2.gas()


  gas:

       temperature   2224.3 K
          pressure   1.0132e+05 Pa
           density   0.15014 kg/m^3
  mean mol. weight   27.405 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.5821e+05       -7.0762e+06  J
   internal energy       -9.3306e+05        -2.557e+07  J
           entropy            9880.3        2.7077e+05  J/K
    Gibbs function       -2.2235e+07       -6.0935e+08  J
 heat capacity c_p            1515.4             41529  J/K
 heat capacity c_v              1212             33214  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                N2           0.72473           0.70897           -27.639
                 H        1.5337e-05        0.00041696           -12.652
                 O         0.0001513        0.00025916           -17.07

In [ ]:
dir(flame)

In [32]:
flame2.X[:,0]

array([ 1.71546589e-028,  7.14932127e-001, -3.55494593e-022,
        2.27525189e-024,  1.10927180e-022,  2.05920637e-021,
        4.42804292e-012,  2.83653691e-018,  2.07552170e-021,
        1.90045249e-001,  5.60368458e-024,  1.40228905e-025,
       -5.64545942e-026, -1.19188528e-035,  1.73687576e-017,
        9.50226244e-002, -2.56209642e-033,  3.69428220e-020,
        1.11336321e-028, -1.92402981e-039,  1.45194623e-023,
        1.68130770e-019,  7.24331073e-022, -3.63351216e-026,
        3.26811440e-023, -1.63217656e-025,  2.71063116e-023,
       -6.80726519e-030, -5.70742685e-018,  1.01480673e-039,
        5.75070019e-027,  4.38563062e-026,  8.73239908e-030,
        1.81888076e-025, -6.84501560e-037, -7.41575878e-027,
       -7.52318842e-026,  9.15957261e-025,  1.12501818e-025,
       -2.17473071e-026, -1.00822411e-025, -5.57588565e-040,
       -4.57293033e-041, -1.15646538e-024,  1.99237450e-025,
       -8.43457100e-025,  5.06220431e-024,  1.07533382e-024,
        7.04324490e-028,

In [33]:
flame2.gas.species_names.index('HF')

116

In [34]:
flame2.X[116,0]

7.695370879977316e-21

In [ ]:
flame.X.shape

In [ ]:
flame.velocity[0]

In [ ]:
# get the top species, put it in a dataframe

In [ ]:
names = flame.gas.species_names
order = np.argsort(flame.X[:,0])

In [ ]:
conc = np.sort(flame.X[:,0])[::-1]
for c in conc:
    print(c)

In [ ]:
conc2 = np.sort(flame2.X[:,0])[::-1]
for c in conc2:
    print(c)

In [ ]:
top_sp = flame.X[order, 0]
for i in range(0, len(top_sp)):
    if top_sp[i] < 0.0:
        top_sp[i] = 0.0

In [ ]:
top_sp

In [ ]:
sorted_names = [i for _, i in sorted(zip(flame.X[:,0], names))]


In [ ]:
sorted_names

In [ ]:
# top 20 species
top50 = sorted_names[-50:]

In [ ]:
top50

In [ ]:
flame.X[20,0]

In [ ]:
# NIST get top species
names_NIST = flame2.gas.species_names
sorted_names_NIST = [i for _, i in sorted(zip(flame2.X[:,0], names_NIST))]

In [ ]:
sorted_names_NIST

In [20]:
# NIST_truncated_flame get top species
names_NIST_truncated = NIST_truncated_flame.gas.species_names
sorted_names_NIST_truncated = [i for _, i in sorted(zip(NIST_truncated_flame.X[:,0], names_NIST_truncated))]

for name in sorted_names_NIST_truncated[::-1]:
    print(name)

N2
O2
CH4
H2
H2O
CO
CH
C
CH2
H2O2
lC5H7
C4H2
C5H4OH
C5H6
C3H8
CH2CHCHCHO
CH3COCH3
CH2OCH2
HO2
CH3CH2CHO
iC4H10
C2H3CHOCH2
C5H4O
CH3OH
C6H4CH3
C6H5CH2OH
l-C6H4
nC4H3
C2H3
C6H6
C4H6-2
CH3CHCHCHO
CH3
C6H2
CH2O
c-C4H5
CH2CHCO
C2H5OH
C2H4
HCCOH
C6H5CO
C4H612
o-C6H4
C6H5CH2
AR
C4H6O25
HCO
CH3O
C5H5O(2,4)
C2H5
CH2*
nC3H7
iC3H7
nC4H5
HCCO
C4H7
pC4H9
C2O
C6H5C2H
H2CC
C4H5-2
iC4H3
CH2OH
sC4H9
CH2OCH
CH3CHOH
tC4H9
iC4H9
CH3CH2O
C5H5O(1,3)
C3H3
CH2CHO
OC6H4CH3
iC4H7
CH3CO
CH3CHO
OH
CH2CO
C6H5O
CH3CHCHCO
HOC2H4O2
HOC6H4CH3
C6H5OH
C4H4O
C2H2
pC3H4
aC3H5
aC3H4
iC4H8
C4H10
C6H3
C2H6
cC3H4
C6H5CH3
C2H
CO2
C4H6
C4H81
C5H5OH
C5H5
CH3CCH2
C4H82
iC4H5
C6H5CHO
C6H4O2
CH3CHCH
C4H6O23
H2C4O
C2H3CHO
C4H4
C3H6
C6H5
CH3CHOCH2
O
C2H4OH
H


In [24]:
for conc in np.sort(NIST_truncated_flame.X[:,0])[::-1]:
    print(conc)

0.7149321266936517
0.19004524886793786
0.09502262443394462
4.465725308442695e-12
2.814788278208606e-18
1.7008575236534864e-19
9.338845230523488e-21
8.795446215295155e-21
8.636218571633905e-21
6.993567077733604e-21
5.2344512365871095e-21
4.509426178970165e-21
4.507926223713893e-21
3.2691289601144146e-21
2.554665544907425e-21
2.4806354883502802e-21
2.063534779831918e-21
1.983061602222348e-21
1.9424407441690035e-21
1.799268833595302e-21
1.5278611607148362e-21
1.0878667774465257e-21
5.790029951830724e-22
5.268313172335509e-22
3.085966849127901e-22
3.0190805203148173e-22
2.9156360356191074e-22
2.82121641284872e-22
2.666630104231687e-22
2.0394692753815454e-22
1.8184083190800924e-22
1.3283160769533471e-22
1.2951730981214014e-22
1.0297525061294883e-22
9.80940881684904e-23
8.860099480123682e-23
8.405782740077224e-23
7.209501267631866e-23
3.864193901185131e-23
2.5021887248433122e-23
1.7472018259067266e-23
3.8895429191891046e-24
6.493694020973988e-25
4.463160716068725e-26
1.996348474254229e-26
1.

In [ ]:
flame.gas.species_names[-1]

In [ ]:
for i in range(0, len(order)):
    print(names[index], flame.X[i,0])

In [ ]:
flame.X[:,0]

In [66]:
flame.velocity[0]

NameError: name 'flame' is not defined

In [ ]:
%matplotlib inline
plt.plot(flame.velocity)
# plt.show()

In [ ]:
dir(flame)

In [ ]:
len(flame.creation_rates)

In [ ]:
flame.gas.TPX

In [ ]:
len(flame.T)

In [ ]:
gas

In [ ]:
gas.TPX = 298.0, ct.one_atm, flame.X[:,0]

In [ ]:
dir(gas)

In [ ]:
gas.reaction_equations()

In [ ]:
# get me top 100 reactions

rates = np.zeros(len(gas.reactions()))
for i in range(0, len(gas.reactions())):
    if gas.forward_rates_of_progress[i] > gas.reverse_rates_of_progress[i]:
        rates[i] = gas.forward_rates_of_progress[i]
    else:
        rates[i] = gas.reverse_rates_of_progress[i]
        
rxn_names = gas.reaction_equations()      
sorted_rxn_names = [i for _, i in sorted(zip(rates, rxn_names))]

In [ ]:
sorted_rates = [i for _, i in sorted(zip(rates, rates))]

In [ ]:
for i in range(0, len(gas.reactions())):
    print(gas.forward_rates_of_progress[i], gas.reverse_rates_of_progress[i])


In [ ]:
for i in range(0, len(sorted_rxn_names)):
    print(sorted_rxn_names[i], sorted_rates[i])

In [ ]:
sorted_rxn_names[1]

In [ ]:
sorted_rxn_names[-2]

In [ ]:
gas.reverse_rates_of_progress[-2]

In [ ]:
np.min(gas.reverse_rates_of_progress)

In [ ]:
len(gas.forward_rates_of_progress)

In [ ]:
gas.reactions()[1]

In [ ]:
gas.reactions()[1].rate(298)

In [ ]:
dir(gas.reactions()[0])

In [ ]:
gas.reactions()[0].reactants

In [ ]:
dir(gas.reactions()[1234])

In [ ]:
reaction_names = []
for i, rxn in enumerate(gas.reactions()):
    if 'Br' in rxn.equation:
        continue
    reaction_names.append(rxn.equation)

In [ ]:
print(reaction_names)

In [16]:
NIST_truncated_flame.velocity[0]

0.40805411806648667

In [67]:
flame2.velocity[0]

0.40837503298115757